In [9]:
%load_ext autoreload
%autoreload 2
%run ./initializer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Path (/Users/rik/Documents/VU/DMT/DataMiningTechniquesA2) already exists in sys.path


In [10]:
import pandas as pd
from lgbm_ranker import LGBMRankerModel, LGBMRanker
from feature_tester import FeatureTester
import numpy as np

In [11]:
train_df = pd.read_parquet("data/training_set_processed.parquet")

In [12]:
lgbm_class_model = LGBMRankerModel(
    df = train_df,              # Only initialized to get column names
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[5],
    n_estimators=1000,
    learning_rate=0.05,
)

In [ ]:
feature_tester = FeatureTester(lgbm_class_model)
list_of_categorical_features_threshold = [{"srch_destination_id": 1000}, {"srch_destination_id": 500}, {"srch_destination_id": 2500}, {"srch_destination_id": 5000}, 
                                          {"site_id": 5000, "visitor_location_country_id": 1000, "prop_country_id": 1000, "srch_destination_id": 5000}, 
                                          {"site_id": 2500, "visitor_location_country_id": 500, "prop_country_id": 500, "srch_destination_id": 5000}, {"site_id": 2500, "visitor_location_country_id": 500, "prop_country_id": 500, "srch_destination_id": 2500}]
feature_tester.test_cardinal_reduction_methods(list_of_categorical_features_threshold)


In [ ]:
categorical_features_threshold = {"site_id": 5000, "visitor_location_country_id": 1000, "prop_country_id": 1000, "srch_destination_id": 5000}
feature_tester = FeatureTester(lgbm_class_model, categorical_features_threshold= categorical_features_threshold)

impute_null_columns_methods = [{"visitor_hist_starrating": "median", "visitor_hist_adr_usd": "mean", "prop_review_score": "median", "orig_destination_distance": "mean", "srch_query_affinity_score":"mean", "prop_log_historical_price":"mean", "prop_location_score2": "mean"},
                               {'prop_review_score': 'median', 'orig_destination_distance': 'mean'}]
feature_tester.test_na_fill_method(impute_null_columns_methods)

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.347239 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6646
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 110
Training until validation scores don't improve for 50 rounds
[20]	valid_0's ndcg@5: 0.366785
[40]	valid_0's ndcg@5: 0.374209
[60]	valid_0's ndcg@5: 0.378155
[80]	valid_0's ndcg@5: 0.381529
[100]	valid_0's ndcg@5: 0.38377
[120]	valid_0's ndcg@5: 0.38651
[140]	valid_0's ndcg@5: 0.387803
[160]	valid_0's ndcg@5: 0.389335
[180]	valid_0's ndcg@5: 0.390733
[200]	valid_0's ndcg@5: 0.392212
[220]	valid_0's ndcg@5: 0.392544
[240]	valid_0's ndcg@5: 0.394064
[260]	valid_0's ndcg@5: 0.394788
[280]	valid_0's ndcg@5

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.398049 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6689
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 110
Training until validation scores don't improve for 50 rounds
[20]	valid_0's ndcg@5: 0.373593
[40]	valid_0's ndcg@5: 0.380066
[60]	valid_0's ndcg@5: 0.383527
[80]	valid_0's ndcg@5: 0.386669
[100]	valid_0's ndcg@5: 0.38862
[120]	valid_0's ndcg@5: 0.390541
[140]	valid_0's ndcg@5: 0.392068
[160]	valid_0's ndcg@5: 0.393553
[180]	valid_0's ndcg@5: 0.395142
[200]	valid_0's ndcg@5: 0.395762
[220]	valid_0's ndcg@5: 0.396536
[240]	valid_0's ndcg@5: 0.397313
[260]	valid_0's ndcg@5: 0.397977
[280]	valid_0's ndcg@

TypeError: cannot unpack non-iterable NoneType object

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.398524 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7548
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 110
Training until validation scores don't improve for 50 rounds
[20]	valid_0's ndcg@5: 0.37273
[40]	valid_0's ndcg@5: 0.38006
[60]	valid_0's ndcg@5: 0.383718
[80]	valid_0's ndcg@5: 0.386887
[100]	valid_0's ndcg@5: 0.389358
[120]	valid_0's ndcg@5: 0.391048
[140]	valid_0's ndcg@5: 0.391497
[160]	valid_0's ndcg@5: 0.392808
[180]	valid_0's ndcg@5: 0.394049
[200]	valid_0's ndcg@5: 0.395328
[220]	valid_0's ndcg@5: 0.39

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.400015 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8181
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 110
Training until validation scores don't improve for 50 rounds
[20]	valid_0's ndcg@5: 0.372528
[40]	valid_0's ndcg@5: 0.379065
[60]	valid_0's ndcg@5: 0.383708
[80]	valid_0's ndcg@5: 0.385855
[100]	valid_0's ndcg@5: 0.388299
[120]	valid_0's ndcg@5: 0.389882
[140]	valid_0's ndcg@5: 0.391303
[160]	valid_0's ndcg@5: 0.392743
[180]	valid_0's ndcg@5: 0.392921
[200]	valid_0's ndcg@5: 0.393923
[220]	valid_0's ndcg@5: 0.

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


...

[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.370680 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6692
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 110
Training until validation scores don't improve for 50 rounds
[20]	valid_0's ndcg@5: 0.373593
[40]	valid_0's ndcg@5: 0.380066
[60]	valid_0's ndcg@5: 0.383527
[80]	valid_0's ndcg@5: 0.3868
[100]	valid_0's ndcg@5: 0.388356
[120]	valid_0's ndcg@5: 0.389888
[140]	valid_0's ndcg@5: 0.391698
[160]	valid_0's ndcg@5: 0.393013
[180]	valid_0's ndcg@5: 0.394814
[200]	valid_0's ndcg@5: 0.395364
[220]	valid_0's ndcg@5: 0.396567
[240]	valid_0's ndcg@5: 0.397176
[260]	valid_0's ndcg@5: 0.39754
[280]	valid_0's ndcg@5:

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.342298 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6729
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 110
Training until validation scores don't improve for 50 rounds
[20]	valid_0's ndcg@5: 0.376093
[40]	valid_0's ndcg@5: 0.381427
[60]	valid_0's ndcg@5: 0.384801
[80]	valid_0's ndcg@5: 0.387389
[100]	valid_0's ndcg@5: 0.389258
[120]	valid_0's ndcg@5: 0.390642
[140]	valid_0's ndcg@5: 0.392359
[160]	valid_0's ndcg@5: 0.393398
[180]	valid_0's ndcg@5: 0.394758
[200]	valid_0's ndcg@5: 0.395881
[220]	valid_0's ndcg@5: 0.396688
[240]	valid_0's ndcg@5: 0.397638
[260]	valid_0's ndcg@5: 0.398303
[280]	valid_0's ndcg

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.684964 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6933
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 110
Training until validation scores don't improve for 50 rounds
[20]	valid_0's ndcg@5: 0.375024
[40]	valid_0's ndcg@5: 0.381041
[60]	valid_0's ndcg@5: 0.384667
[80]	valid_0's ndcg@5: 0.387697
[100]	valid_0's ndcg@5: 0.390256
[120]	valid_0's ndcg@5: 0.391964
[140]	valid_0's ndcg@5: 0.393238
[160]	valid_0's ndcg@5: 0.394385
[180]	valid_0's ndcg@5: 0.395393
[200]	valid_0's ndcg@5: 0.396394
[220]	valid_0's ndcg@5: 0.